In [ ]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import graphviz 
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Cleaning Section

In [ ]:
table = pd.read_csv("sales_data_sample.csv",encoding='latin-1')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
table.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [ ]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2      302 non-null    object 


In [ ]:
table.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [ ]:
table = table.replace('NaN','0')

Checking duplicates

In [ ]:
table.duplicated().sum()

0

# checking null values

In [ ]:
table.isna().sum()   

ORDERNUMBER            0
QUANTITYORDERED        0
PRICEEACH              0
ORDERLINENUMBER        0
SALES                  0
ORDERDATE              0
STATUS                 0
QTR_ID                 0
MONTH_ID               0
YEAR_ID                0
PRODUCTLINE            0
MSRP                   0
PRODUCTCODE            0
CUSTOMERNAME           0
PHONE                  0
ADDRESSLINE1           0
ADDRESSLINE2        2521
CITY                   0
STATE               1486
POSTALCODE            76
COUNTRY                0
TERRITORY           1074
CONTACTLASTNAME        0
CONTACTFIRSTNAME       0
DEALSIZE               0
dtype: int64

check if this data contain any rows that have more than 4 null values in the same row

In [ ]:
rows = table.isna().sum(axis=1)
print('we have ', rows[rows >= 4].shape[0], ' rows that contian more than 4 null values')
rows[rows >= 4]

we have  0  rows that contian more than 4 null values


Series([], dtype: int64)

checking the data percentage of null values in each coloum

In [ ]:
np.round((table.isna().sum()/table.shape[0])*100)

ORDERNUMBER          0.0
QUANTITYORDERED      0.0
PRICEEACH            0.0
ORDERLINENUMBER      0.0
SALES                0.0
ORDERDATE            0.0
STATUS               0.0
QTR_ID               0.0
MONTH_ID             0.0
YEAR_ID              0.0
PRODUCTLINE          0.0
MSRP                 0.0
PRODUCTCODE          0.0
CUSTOMERNAME         0.0
PHONE                0.0
ADDRESSLINE1         0.0
ADDRESSLINE2        89.0
CITY                 0.0
STATE               53.0
POSTALCODE           3.0
COUNTRY              0.0
TERRITORY           38.0
CONTACTLASTNAME      0.0
CONTACTFIRSTNAME     0.0
DEALSIZE             0.0
dtype: float64

Drop the coloums that exceeded our 50% threshold

In [ ]:
table.drop("ADDRESSLINE1", axis=1, inplace=True)

In [ ]:
table.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,PHONE,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,2125557818,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,26.47.1555,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,+33 1 46 62 7555,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,6265557265,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,6505551386,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


replacing null values in coloum state and TERRITORY and postalcode with mode

In [ ]:
mask = table['STATE'].isna() | table['TERRITORY'].isna() | table['POSTALCODE'].isna()
table[mask].head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,PHONE,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,2125557818,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,26.47.1555,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,+33 1 46 62 7555,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,6265557265,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,6505551386,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [ ]:
table = table.fillna({
    'STATE': table['STATE'].mode()[0],
    'TERRITORY': table['TERRITORY'].mode()[0],
    'POSTALCODE': table['POSTALCODE'].mode()[0],
})

In [ ]:
table.isna().sum()

ORDERNUMBER            0
QUANTITYORDERED        0
PRICEEACH              0
ORDERLINENUMBER        0
SALES                  0
ORDERDATE              0
STATUS                 0
QTR_ID                 0
MONTH_ID               0
YEAR_ID                0
PRODUCTLINE            0
MSRP                   0
PRODUCTCODE            0
CUSTOMERNAME           0
PHONE                  0
ADDRESSLINE2        2521
CITY                   0
STATE                  0
POSTALCODE             0
COUNTRY                0
TERRITORY              0
CONTACTLASTNAME        0
CONTACTFIRSTNAME       0
DEALSIZE               0
dtype: int64

# Checking Outliers

## First we need to describe data

In [ ]:
table.describe()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,QTR_ID,MONTH_ID,YEAR_ID,MSRP
count,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.000000,2823.00000,2823.000000
mean,10258.725115,35.092809,83.658544,6.466171,3553.889072,2.717676,7.092455,2003.81509,100.715551
std,92.085478,9.741443,20.174277,4.225841,1841.865106,1.203878,3.656633,0.69967,40.187912
min,10100.000000,6.000000,26.880000,1.000000,482.130000,1.000000,1.000000,2003.00000,33.000000
25%,10180.000000,27.000000,68.860000,3.000000,2203.430000,2.000000,4.000000,2003.00000,68.000000
50%,10262.000000,35.000000,95.700000,6.000000,3184.800000,3.000000,8.000000,2004.00000,99.000000
75%,10333.500000,43.000000,100.000000,9.000000,4508.000000,4.000000,11.000000,2004.00000,124.000000
max,10425.000000,97.000000,100.000000,18.000000,14082.800000,4.000000,12.000000,2005.00000,214.000000


## Ploting data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
cols = ['ORDERNUMBER' , 'QUANTITYORDERED' , 'PRICEEACH' , 'ORDERLINENUMBER' ,'SALES' ,'QTR_ID' , 'MONTH_ID' , 'YEAR_ID' ,'MSRP'  ]
for col in cols:
sns.boxplot(data = table , y= col)
plt.show()
  

IndentationError: ignored

## Handle outliers

   *After describing and ploting data , we conclude that 'sales' column has outliers*



### Z_score

In [ ]:
from scipy import stats

z_sales = np.abs(stats.zscore(table['SALES']))

threshold = 2.2 #standard cutoff value
table.loc[z_sales > threshold, 'SALES'] = np.nan

table.isna().sum()

In [ ]:
table = table.dropna()
sns.boxplot(data = table , x= 'SALES')
plt.show()

# Visualization

In [ ]:
# table['ORDERDATE'] = table['ORDERDATE'].apply(lambda x :x[4:9])
# table['ORDERDATE']

In [ ]:
table.to_csv('sales_data_sample_cleaned.csv')

In [ ]:
import plotly.express as px
px.bar(table, y='SALES', x=table["ORDERDATE"].astype(str).str.slice(0, 2)+'/'+table["YEAR_ID"].astype(str).str.slice(0, 4), title='sales across year')

In [ ]:
px.bar(table, y='SALES', x='COUNTRY', title='sales across countries')

## Sales for each product line across Months

In [ ]:
import plotly.graph_objects as go 
fig = go.Figure()
# get values of our product lines
product_ lines = table['PRODUCTLINE'].unique()
for line in product_lines:
  # subset tabel
  new_tabel = table[table['PRODUCTLINE'] == line]
  fig.add_trace(
      go.Bar(
          x = new_tabel['MONTH_ID'] , y=new_tabel['SALES'] , name = line
      )
  )
# Create a custom buttons to update between product lines
line_buttons = [
     {
        'label':'All',
        'method': 'update',
        'args' : [{'visible':[True , True, True ,True , True , True , True]},{"title": "Sales for each product line across Months"}]
    },
    {
        'label':'Motorcycles',
        'method': 'update',
        'args' : [{'visible':[True , False, False ,False , False , False , False]},{"title": "Motorcycles Line"}]
    },
     {
        'label':'Classic Cars',
        'method': 'update',
        'args' : [{'visible':[False , True, False ,False , False , False , False]},{"title": "Classic Cars Line"}]
    },
     {
        'label':'Trucks and Buses',
        'method': 'update',
        'args' : [{'visible':[False , False, True ,False , False , False , False]},{"title": "Trucks and Buses"}]
    },
     {
        'label':'Vintage Cars',
        'method': 'update',
        'args' : [{'visible':[False , False, False ,True , False , False , False]},{"title": "Vintage cars"}]
    },
     {
        'label':'Planes',
        'method': 'update',
        'args' : [{'visible':[False , False, False ,False , True , False , False]},{"title": "Planes"}]
    },
    {
        'label':'Ships',
        'method': 'update',
        'args' : [{'visible':[False , False, False ,False , False , True , False]},{"title": "Ships"}]
    },
    {
        'label':'Trains',
        'method': 'update',
        'args' : [{'visible':[False , False, False ,False , False , False , True]},{"title": "Trains"}]
    }
]
fig.update_layout({
    'updatemenus':[
        {
            'type':'buttons',
             'direction':'down',
              'x' : -0.1 , 'y':0.9,
              'showactive':True,
              'active':0,
              'buttons':line_buttons
        }
        
    ]
})
# fig.data[0].visible = False
# fig.data[1].visible = False
# fig.data[2].visible = False
# fig.data[3].visible = False
# fig.data[4].visible = False
# fig.data[5].visible = False
# fig.data[6].visible = False
fig.show()

## Sales across countries for each product

In [ ]:
fig = go.Figure()
# get values of our countries
Country = table['COUNTRY'].unique()
for country in Country:
  # subset tabel
  new_tabel = table[table['COUNTRY'] == country]
  fig.add_trace(
      go.Bar(
          x = new_tabel['PRODUCTLINE'] , y=new_tabel['SALES'] , name = country
      )
  )
  # Create a custom buttons to update between product lines
line_buttons = [
     {
        'label':'All',
        'method': 'update',
        'args' : [{'visible':[True , True, True ,True , True ]},{"title": "Sales across countries for each product"}]
    },
    {
        'label':'Australia',
        'method': 'update',
        'args' : [{'visible':[True , False, False ,False , False]},{"title": "Sales in Australia"}]
    },
     {
        'label':'USA',
        'method': 'update',
        'args' : [{'visible':[False , True, False ,False , False]},{"title": "Sales in USA"}]
    },
     {
        'label':'Singapore',
        'method': 'update',
        'args' : [{'visible':[False , False, True ,False , False]},{"title": "Sales in Singapore"}]
    },
     {
        'label':'Ireland',
        'method': 'update',
        'args' : [{'visible':[False , False, False ,True , False ,]},{"title": "Sales in Ireland"}]
    }
]
fig.update_layout({
    'updatemenus':[
        {
            'type':'buttons',
             'direction':'down',
              'x' : -0.1 , 'y':0.9,
              'showactive':True,
              'active':0,
              'buttons':line_buttons
        }
        
    ]
})
fig.show()

### Sales for each city

In [ ]:

px.bar(table, x='CITY', y='SALES', title='Sales for each city')

In [ ]:
px.bar(table, x='COUNTRY', y='DEALSIZE', title='sales for each company')

In [ ]:
px.bar(table, x='PRODUCTCODE', y='SALES', title='Sales of each product')


In [ ]:
px.sunburst(table, path=['YEAR_ID', 'PRODUCTLINE'], values='QUANTITYORDERED', title='Quantity ordered for each product line in all years')

In [ ]:
!pip install -q dash dash_bootstrap_components jupyter_dash

In [ ]:
Sales_par = px.bar(table, x='PRODUCTCODE', y='SALES', title='Sales of each product')
sales_along_year=px.bar(table, y='SALES', x=table["ORDERDATE"].astype(str).str.slice(0, 2)+'/'+table["YEAR_ID"].astype(str).str.slice(0, 4), title='sales across year')
Sales_of_each_country=px.bar(table, x='COUNTRY', y='SALES', title='sales for each country')
sales_lines_year=px.sunburst(table, path=['YEAR_ID', 'PRODUCTLINE'], values='QUANTITYORDERED', title='Quantity ordered for each product line in all years')
Sales_of_each_city=px.bar(table, x='CITY', y='SALES', title='Sales for each city')

  

In [ ]:
sale_par_fig=dcc.Graph(
    id="sales_par_fig",
    figure=Sales_par
)

sales_along_year_fig=dcc.Graph(
    id='sales_along_year_fig',
    figure=sales_along_year
)
Sales_of_each_country_fig=dcc.Graph(
    id='Sales_of_each_country_fig',
    figure=Sales_of_each_country
)
sales_lines_year_fig=dcc.Graph(
    id='sales_lines_year_fig',
    figure=sales_lines_year
)
Sales_of_each_city_fig=dcc.Graph(
    id='Sales_of_each_city_fig',
    figure=Sales_of_each_city
)
app = JupyterDash('Basic Dash Dashboard', 
                   external_stylesheets=[dbc.themes.BOOTSTRAP], 
                   title="sales data sample Dashboard"
 )
app.layout = dbc.Container(
     [
      dbc.Row(
          [
           dbc.Col(html.H1('sales Dashboard Analysis'))
          ]
      ),
      dbc.Row(
          [
           dbc.Row(
               [
                dbc.Col(dbc.Card(sale_par_fig)),
      
               ],
               className='my-2'
           ),
           dbc.Row(
               [
                dbc.Col(dbc.Card(Sales_of_each_city_fig), md=6),
                dbc.Col(dbc.Card(Sales_of_each_country_fig), md=6),
               ]
           )
          ],
          className='my-3'

      ),
      dbc.Row(
          [
           dbc.Col(dbc.Card(sales_lines_year_fig), md=12),
          
          ]
      ),
      dbc.Row(
          [
           dbc.Col(dbc.Card(sales_along_year_fig), md=12),
          
          ],
          className='my-3'
      )
     ]
 )

app.run_server(debug=True)

# Modelling Phase

## Regression

In [ ]:
table.head()

## Regression in one variable

In [ ]:
plt.scatter(x=table['QUANTITYORDERED'], y=table['SALES'])
plt.show()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(table[['QUANTITYORDERED']], table[['SALES']], test_size=0.2)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
model.coef_ # a

In [ ]:
model.intercept_ # b

In [ ]:
model.score(x_train, y_train)

In [ ]:
model.score(x_test, y_test)

In [ ]:
a = model.coef_[0, 0]
b = model.intercept_[0]
x = np.arange(0, 50)
y = a*x + b
plt.scatter(x=table['QUANTITYORDERED'], y=table['SALES'])
plt.plot(x, y, 'r--')
plt.show()

# Classification

In [ ]:
table.head()

In [ ]:
features=table[['QUANTITYORDERED','SALES','PRICEEACH','PRODUCTLINE']]

In [ ]:
features.head()

In [ ]:
sns.pairplot(features, hue='PRODUCTLINE')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    table[['QUANTITYORDERED', 'SALES']], 
    table[['PRODUCTLINE']], 
    test_size=0.2,
    shuffle=True
)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

In [ ]:
dot_data = export_graphviz(
    model,
    feature_names=['QUANTITYORDERED', 'SALES'],  
    class_names=table['PRODUCTLINE'].unique(),  
    filled=True,
    rounded=True,  
    special_characters=True
)  
graph = graphviz.Source(dot_data)  
graph 

In [ ]:
table['PRODUCTLINE'].unique()

In [ ]:
model.score(x_train, y_train)

In [ ]:
model.score(x_test, y_test)

## Fixing overfitting

In [ ]:
model = DecisionTreeClassifier(max_depth=4, min_samples_split=6)
model.fit(x_train, y_train)

In [ ]:
dot_data = export_graphviz(
    model,
    feature_names=['QUANTITYORDERED', 'SALES'],  
    class_names=table['PRODUCTLINE'].unique(),  
    filled=True,
    rounded=True,  
    special_characters=True
)  
graph = graphviz.Source(dot_data)  
graph

In [ ]:
model.score(x_train, y_train)

In [ ]:
model.score(x_test, y_test)